# Keras Overview

In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10))

In [4]:
type(model)

tensorflow.python.keras.engine.sequential.Sequential

### Train and evaluate

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
              run_eagerly=True
             )

### Trian from Numpy data

In [15]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=3, batch_size=32)

Train on 1000 samples
Epoch 1/3
1000/1000 [==============================] - 1s 606us/sample - loss: 127980.7010 - accuracy: 0.1070
Epoch 2/3
1000/1000 [==============================] - 0s 371us/sample - loss: 114634.6733 - accuracy: 0.0870
Epoch 3/3
1000/1000 [==============================] - 0s 383us/sample - loss: 111367.7010 - accuracy: 0.1090


### Train from tf.data datasets

In [18]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)
model.fit(dataset, epochs=3)

Train for 32 steps
Epoch 1/3
32/32 [==============================] - 1s 43ms/step - loss: 139176.3711 - accuracy: 0.0880
Epoch 2/3
32/32 [==============================] - 1s 16ms/step - loss: 166031.9279 - accuracy: 0.0980
Epoch 3/3
32/32 [==============================] - 1s 21ms/step - loss: 162634.1485 - accuracy: 0.0870


### Evaluate and predict

In [21]:
# With Numpy arrays
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

# With a Dataset
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.evaluate(dataset)

32/32 [==============================] - 0s 6ms/step - loss: 136551.1450 - accuracy: 0.0940


[136551.14501953125, 0.094]

In [22]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


### The Functional API

In [29]:
inputs = tf.keras.Input(shape=(32,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = tf.keras.layers.Dense(10)(x)

In [34]:
# Instantiate the model given inputs and outputs.

model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 372us/sample - loss: 14.3918 - accuracy: 0.1180
Epoch 2/5
1000/1000 [==============================] - 0s 47us/sample - loss: 29.9525 - accuracy: 0.1120
Epoch 3/5
1000/1000 [==============================] - 0s 57us/sample - loss: 56.3153 - accuracy: 0.1000
Epoch 4/5
1000/1000 [==============================] - 0s 48us/sample - loss: 88.3219 - accuracy: 0.1000
Epoch 5/5
1000/1000 [==============================] - 0s 47us/sample - loss: 127.2526 - accuracy: 0.1010


### Model Subclassing

In [39]:
class MyModel(tf.keras.Model):
    
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # define the layers
        self.dense_1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(num_classes)
    
    def call(self, inputs):
        # define the forward pass
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [41]:
model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
             )
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 304us/sample - loss: 12.2240 - accuracy: 0.1090
Epoch 2/5
1000/1000 [==============================] - 0s 56us/sample - loss: 13.6469 - accuracy: 0.0930
Epoch 3/5
1000/1000 [==============================] - 0s 44us/sample - loss: 14.7312 - accuracy: 0.0940
Epoch 4/5
1000/1000 [==============================] - 0s 44us/sample - loss: 15.3373 - accuracy: 0.0940
Epoch 5/5
1000/1000 [==============================] - 0s 44us/sample - loss: 16.1693 - accuracy: 0.0950


### Custom Layers

In [43]:
# Example of how to build a cutom layer.

class MyLayer(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        # create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True
                                     )
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        returncls(**config)

In [44]:
# create a model using the custom layer
model = tf.keras.Sequential([MyLayer(10)])
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=3)

Train on 1000 samples
Epoch 1/3
1000/1000 [==============================] - 0s 211us/sample - loss: 11.4913 - accuracy: 0.1050
Epoch 2/3
1000/1000 [==============================] - 0s 48us/sample - loss: 11.4927 - accuracy: 0.1050
Epoch 3/3
1000/1000 [==============================] - 0s 47us/sample - loss: 11.4922 - accuracy: 0.1040
